In [17]:
# Suppress deprecation warnings
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)


import pandas as pd
pd.set_option('display.max_colwidth', 0)


# Update the path with the actual path to your folder
import sys
library_path = 'my_repo/v31_apriori'
sys.path.append(library_path)

from V31Apriori import V31
import json
from estnltk import Text
import os
import re

In [2]:
SQLITEFILE = "v31data.db"

In [3]:
v31 = V31(
    SQLITEFILE,
    form_treshold_percent = 20,
    form_treshold_count = 5,
    apriori_min_support=0.05,
    apriori_treshold_delta = 0.03,
    apriori_treshold_percent=50,
    datarows_treshold=100000
    )

In [36]:
# reading verb pattern files:

df_len1 = pd.read_csv('data/verb_patterns_len1.csv')
df_len2 = pd.read_csv('data/verb_patterns_len2.csv')

In [37]:
df_len1

,Unnamed: 0,word,government_with_case,word_1,word_2,word_3
0,0,aasima,p,p,NaN,NaN
1,1,aasima,g kallal,g,kallal.adp,NaN
2,2,abielluma,kom,kom,NaN,NaN
3,3,abikätt ulatama,all,all,NaN,NaN
4,4,abstraheeruma,el,el,NaN,NaN
...,...,...,...,...,...,...
2533,2533,ümmardama,p,p,NaN,NaN
2534,2534,üritama,p teha,p,teha.v,NaN
2535,2535,ütlema,all,all,NaN,NaN
2536,2536,ütlema,g kohta,g,kohta.adp,NaN


In [38]:
df_len2

,Unnamed: 0,word,government_with_case,member1_wrd1,member1_wrd2,member2_wrd1,member2_wrd2
0,0,abistama,p + in,p,NaN,in,NaN
1,1,aitama,ad + p teha,ad,NaN,p,teha.v
2,2,ette heitma,all + p,all,NaN,p,NaN
3,3,informeerima,p + el,p,NaN,el,NaN
4,4,jagama,p + kom,p,NaN,kom,NaN
5,5,keelama,ad + p teha,ad,NaN,p,teha.v
6,6,kleepima,all + g peale,all,NaN,g,peale.adp
7,7,kohtlema,p + kuidas,p,NaN,kuidas.?,NaN
8,8,kokku leppima,kom + in,kom,NaN,in,NaN
9,9,käskima,ad + p teha,ad,NaN,p,teha.v


In [39]:
df_len1 = df_len1.fillna('')
df_len2 = df_len2.fillna('')

In [40]:
# getting transactions of each (unique) verb, transactions will be saved in json files

unique_1 = list(df_len1['word'].unique())
unique_2 = list(df_len2['word'].unique())

In [41]:
len(unique_1)

1855

In [42]:
len(unique_2)

31

In [43]:
unique = sorted(list(set(unique_1+unique_2)))

In [44]:
len(unique)

1868

In [ ]:
# kui ühe transaktsiooni wall time on u 3 minutit, siis 1868 transaktsiooni tegemine võib võtta aega 90 tunni ringis
# esialgu proovin 10 verbiga, jättes välja sellised, kus compound pikkus > 2 (nt 'altari ette viima')

In [12]:
unique_10 = [unique[i] for i in range(10) if len(unique[i].split()) < 3]

In [13]:
unique_10

['aasima',
 'abielluma',
 'abikätt ulatama',
 'abistama',
 'abstraheeruma',
 'adresseerima',
 'aevastama',
 'agiteerima',
 'ahistama',
 'ahvima']

In [72]:
%%time

v = 'vastu võtma koos'
verb_compound = [] # TODO: küsida üle, mida teha, kui verbil on compound osad (1 või enam)
verb = ''
splitted = v.split()
if len(splitted) > 1:
    for i in range(len(splitted)):
        if 'V' in Text(splitted[i]).tag_layer('morph_analysis').morvastu, koosph_analysis.partofspeech[0]:
            verb = splitted[i]
        else:
            verb_compound.append(splitted[i])
transactions = v31.get_transactions(verb=verb, verb_compound=', '.join(verb_compound))
raw_transactions = v31._raw_transactions
    

CPU times: total: 25.5 s
Wall time: 3min 16s


In [71]:
display(raw_transactions)

{}

In [66]:
%%time

v = 'alla hüppama'
verb_compound = [] # kui compound osi rohkem kui 1, siis eraldatakse komaga
verb = ''
splitted = v.split()
if len(splitted) > 1:
    for i in range(len(splitted)):
        if 'V' in Text(splitted[i]).tag_layer('morph_analysis').morph_analysis.partofspeech[0]:
            verb = splitted[i]
        else:
            verb_compound.append(splitted[i])
transactions = v31.get_transactions(verb=verb, verb_compound=','.join(verb_compound))
raw_transactions = v31._raw_transactions

CPU times: total: 23 s
Wall time: 3min 8s


In [67]:
display(raw_transactions)

{16176: [{'deprel': 'OBL',
   'case': 'in',
   'feats': 'com,in,sg',
   'frequent_form': 'hädas',
   'obl_case': '',
   'loc': 3},
  {'deprel': 'NSUBJ',
   'case': 'nom',
   'feats': 'com,nom,pl',
   'frequent_form': 'poisid',
   'obl_case': '',
   'loc': 5},
  {'deprel': 'OBL',
   'case': 'el',
   'feats': 'com,el,sg',
   'frequent_form': 'lakaluugist',
   'obl_case': '',
   'loc': 6},
  {'deprel': 'OBL',
   'case': 'ill',
   'feats': 'com,ill,pl',
   'frequent_form': 'kasvuhooneklaasidesse',
   'obl_case': '',
   'loc': 8}],
 20021: [{'deprel': 'OBL',
   'case': 'kom',
   'feats': 'com,kom,sg',
   'frequent_form': 'deltaplaaniga',
   'obl_case': '',
   'loc': 2},
  {'deprel': 'OBL',
   'case': 'abl',
   'feats': 'abl,com,sg',
   'frequent_form': 'kaljult',
   'obl_case': '',
   'loc': 3},
  {'deprel': 'ADVCL',
   'case': '',
   'feats': 'af,aux,impf,indic,ps,ps3,sg',
   'frequent_form': 'muutis',
   'obl_case': '',
   'loc': 8}],
 167405: [{'deprel': 'ADVMOD',
   'case': '',
   'feat

In [14]:
%%time

for v in unique_10:
    verb_compound = '' # kui compound osi rohkem kui 1, siis eraldatakse komaga
    verb = ''
    splitted = v.split()
    if len(splitted) > 1:
        for i in range(len(splitted)):
            if 'V' in Text(splitted[i]).tag_layer('morph_analysis').morph_analysis.partofspeech[0]:
                verb = splitted[i]
            else:
                verb_compound = splitted[i]
    else:
        verb = v
    transactions = v31.get_transactions(verb=verb, verb_compound=verb_compound)
    raw_transactions = v31._raw_transactions
    splitted_joined = '_'.join(splitted)
    with open(f'transactions/raw_trans_{splitted_joined}.json', 'w') as f: 
        json.dump(raw_transactions, f)
    

CPU times: total: 4min 6s
Wall time: 33min


In [57]:
# mustrite otsing ja esinemissageduste leidmine

path = 'transactions/'

len1_grouped = df_len1.groupby('word') # grouping both dataframes by verbs
len2_grouped = df_len2.groupby('word')

case_mapping = {'nom': 'nom',  # nimetav
                'g': 'gen',  # omastav
                'p': 'part',  # osastav
                'adt': 'adit',  # lyh sisse
                'ill': 'ill',  # sisse
                'in': 'in',  # sees
                'el': 'el',  # seest
                'all': 'all',  # alale
                'ad': 'ad',  # alal
                'abl': 'abl',  # alalt
                'tr': 'tr',  # saav
                'ter': 'term',  # rajav
                'es': 'es',  # olev
                'ab': 'abes',  # ilma
                'kom': 'kom'}  # kaasa

for filename in os.listdir(path):
    raw_transactions = ''
    verb = ' '.join(re.sub(r'raw_trans_(\D+).json', r'\1', filename).split('_'))
    
    with open(os.path.join(path, filename)) as f:
        raw_transactions = json.load(f)
    
    result = dict()
    for name, group in len1_grouped:
        if name == verb:
            pattern_freq_list = []
            
            for idx, row in group.iterrows():
                freq = 0
                pattern_members = [row['word_1'].split('.'), row['word_2'].split('.'), row['word_3'].split('.')]
                
                if '?' in pattern_members[0] or '?' in pattern_members[1] or '?' in pattern_members[2]:
                    continue # esialgu ignoreeritakse sõnu, mis pole ei kääne, verb ega kaassõna
                    
                meaning = []
                pattern = []
                examples = []
                
                for member in pattern_members:
                    if len(member) == 1:
                        if member[0] == '':
                            continue
                        else:
                            meaning.append('case')
                            pattern.append(member[0])
                    else:
                        if member[1] == 'v':
                            meaning.append('verb')
                            pattern.append(member[0])
                        else:
                            meaning.append('adp')
                            pattern.append(member[0])
                
                for key, kids in raw_transactions.items():
                    found_pattern = [False]*len(pattern)
                    freq_forms = []
                    
                    for i in range(len(pattern)):
                        for kid in kids:
                            if kid['deprel'] == 'OBL' and meaning[i] == 'case':
                                if kid['case'] == case_mapping[pattern[i]]:
                                    found_pattern[i] = True
                                    freq_forms.append(kid['frequent_form'])
                            elif meaning[i] == 'verb':
                                if 'sup' in kid['feats'].split(',') or 'inf' in kid['feats'].split(','):
                                    found_pattern[i] = True
                                    freq_forms.append(kid['frequent_form'])
                            else:
                                if kid['obl_case'] == pattern[i]:
                                    found_pattern[i] = True
                                    freq_forms.append(kid['obl_case'])
                    if all(found_pattern):
                        freq+=1
                        examples.append(freq_forms)
                        
                pattern_freq_list.append([' '.join(pattern), [' '.join(e) for e in examples][:3], freq]) # pattern, frequent forms, frequency
                
            result[verb] = pattern_freq_list
            

    print(result)

{'aasima': [['p', ['aastat', 'karja', 'lihvimist'], 3], ['g kallal', [], 0]]}
{'abielluma': [['kom', ['temaga', 'mehega', 'meestega'], 1607]]}
{'abikätt ulatama': [['all', [], 0]]}
{}
{'abstraheeruma': [['el', ['tegelikkusest', 'arvamusartiklist', 'nimest'], 3]]}
{'adresseerima': [['p', ['seda'], 1], ['all', ['abistajatele', 'haridusministrile', 'persoonile'], 83]]}
{'aevastama': [['g peale', [], 0]]}
{'agiteerima': [['p', ['päeva üldkoosolekut', 'laulu', 'aega'], 7]]}
{'ahistama': [['p', ['aastakümneid', 'lahkuminekut', 'aastat'], 31]]}
{'ahvima': [['p', [], 0]]}
